In [1]:
#import igraph as ig
import gravis as gv
import networkx as nx


# Attach dir one above to handle IDE subdir
import sys, os
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

sys.path.append("/home/tomk/Documents/Coding/gitHub/datamodels/src")
print(sys.path)
from rdflib import URIRef, Graph, Namespace, Literal
from rdflib.term import BNode

['/home/tomk/Documents/Coding/gitHub/datamodels/IDE', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/tomk/environment/devpy/lib/python3.10/site-packages', '/home/tomk/Documents/Coding/gitHub/daglit/src', '/home/tomk/Documents/Coding/gitHub/datamodels/src', '/home/tomk/Documents/Coding/gitHub/datamodels/src']


In [2]:
g = Graph()

g.parse ('/home/tomk/Documents/Coding/gitHub/datamodels/src/models/DMEAR/datamodels_rdf.owl', format='xml')
#g.parse ('/home/tomk/Documents/Coding/gitHub/datamodels/src/models/DMEAR/DMEAR_ser.rdf', format='xml')
#g.parse ('/home/tomk/Documents/Coding/gitHub/datamodels/src/models/in-development/EntArchModel.owl', format='xml')
#g.parse ('/home/tomk/Documents/Coding/gitHub/datamodels/src/models/in-development/archi_test.owl', format='xml')
#g.parse ('/home/tomk/Documents/Coding/gitHub/datamodels/src/models/models/core/discourse/Discourse.owl', format='xml')
#g.parse ('/home/tomk/Documents/Coding/gitHub/datamodels/src/models/core/serialization/Serialization.owl', format='xml')
#g.parse ('/home/tomk/Documents/Coding/gitHub/datamodels/src/models/sample/sample_ontology.owl', format='xml')
#g.parse ('/home/tomk/Documents/Coding/gitHub/datamodels/src/models/hancestro.owl', format='xml')


<Graph identifier=N7282a7b6627e4f6ab33a0ae39585097a (<class 'rdflib.graph.Graph'>)>

In [17]:
def isURIRef(r):
    return isinstance(r, URIRef)

def isBNode(r):
    return isinstance(r, BNode)

def get_concrete_owl_classes(owl_graph):
    return filter(isURIRef, (s for s,p,o in owl_graph.triples((None, None, URIRef('http://www.w3.org/2002/07/owl#Class')))))

def get_concrete_owl_object_properties(owl_graph):
    return filter(isURIRef, (s for s,p,o in owl_graph.triples((None, None, URIRef('http://www.w3.org/2002/07/owl#ObjectProperty')))))

def get_concrete_owl_data_properties(owl_graph):
    return filter(isURIRef, (s for s,p,o in owl_graph.triples((None, None, URIRef('http://www.w3.org/2002/07/owl#DatatypeProperty')))))

def get_class_subclass_relations(owl_graph):
    return ((s,o) for s,p,o in owl_graph.triples((None, URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),  None)))

def process_bnode(owl_graph, bnode):
    #print(bnode)
    #print(list(owl_graph.triples((bnode, None, None))))
    # Look for contents that might be expressed using union/intersection or complement
    i_u_c = set([URIRef('http://www.w3.org/2002/07/owl#unionOf'), 
                URIRef('http://www.w3.org/2002/07/owl#intersectionOf'), 
                URIRef('http://www.w3.org/2002/07/owl#complementOf'), 
                URIRef('http://www.w3.org/2002/07/owl#oneOf')])
    if any((p in i_u_c for s,p,o in owl_graph.triples((bnode, None, None)))):
        u_pointers = set((o for s,p,o in owl_graph.triples((bnode, URIRef('http://www.w3.org/2002/07/owl#unionOf'), None))))
        i_pointers = set((o for s,p,o in owl_graph.triples((bnode, URIRef('http://www.w3.org/2002/07/owl#intersectionOf'), None))))
        o_pointers = set((o for s,p,o in owl_graph.triples((bnode, URIRef('http://www.w3.org/2002/07/owl#oneOf'), None))))
        if len (u_pointers)>0:
            union_nodes = get_rdf_collection(owl_graph, u_pointers.pop())
            return union_nodes
        elif len (i_pointers)>0:
            intersection_nodes=get_rdf_collection(owl_graph, i_pointers.pop())
            return intersection_nodes
        elif len (o_pointers)>0:
            oneof_nodes=get_rdf_collection(owl_graph, o_pointers.pop())
            return oneof_nodes
        

    
def get_node_label(owl_graph, node, resolvers=None):
    labels = set([o for s,p,o in owl_graph.triples((node, URIRef('http://www.w3.org/2000/01/rdf-schema#label'),None))])
    if len(labels)>0:
        return labels.pop().toPython()
    else:
        if resolvers is not None:
            return resolve_uri_label(node.toPython(), resolvers)
        return node.toPython()

def get_node_literals(owl_graph, node, resolvers=None):
    literals = set([(p,o) for s,p,o in owl_graph.triples((node, None,None)) if isinstance(o,Literal)])
    if len(literals)>0:
        return {resolve_uri_label(k, resolvers) : str(v) for k,v in literals}
    

def dict_to_html_table(kv_dict):
    if kv_dict is None:
        kv_dict={}
    rows = ""
    for k,v in kv_dict.items():
        rows = rows + f"""<tr><td>{k}</td><td>{v}</td></tr>"""
    table = f"""<table width="95%"><tr>
    <th width="20%">Key</th><th width="80%">Value</th>
    </tr>
    {rows}
    </table>"""
    return table
    
def resolve_uri_label(label, resolvers=None):
    if isinstance(label, (URIRef,Literal)):
        label=label.toPython()
    if resolvers is None:
        resolvers={}
    for k,v in resolvers.items():
        if v in label:
            return label.replace(v,k+":")
        
    return label




def get_rdf_collection(owl_graph, collection_pointer, collection=None):
    #print(collection_pointer)
    if collection is None:
        collection=set()
    try:
        first = set((o for s,p,o in owl_graph.triples((collection_pointer, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first'), None)))).pop()
    except KeyError:
    
        return collection
    collection.add(first)
    try:
        rest = set((o for s,p,o in owl_graph.triples((collection_pointer, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#rest'), None)))).pop()
    except KeyError:
    
        return collection
    
    if rest == URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#nil'):
    
        return collection
    else:
    
        get_rdf_collection(owl_graph, rest, collection=collection)
    
    return collection

def get_orientable_owl_object_properties(owl_graph, p_list):
    d_classes = []
    for p in p_list:
        d_set = set((d for pr,x,d in owl_graph.triples((p, URIRef('http://www.w3.org/2000/01/rdf-schema#domain'), None))))

        r_set = set((r for pr,x,r in owl_graph.triples((p, URIRef('http://www.w3.org/2000/01/rdf-schema#range'), None))))

        if any((isBNode(d) for d in d_set)):
            dx_set = set()
            for d in d_set:
                if isBNode(d):
                    dx_set.update(process_bnode(owl_graph,d ))
            #dx_set = [process_bnode(owl_graph, d) for d in d_set if isBNode(d)]
        else:
            dx_set = d_set
        if any((isBNode(r) for r in r_set)):
            #print("!", r_set)
            rx_set = set()
            for r in r_set:
                if isBNode(r):
                    #print("!B!", r, rx_set, process_bnode(owl_graph, r))
                    rx_set.update(process_bnode(owl_graph, r))
                    
                    if all([isinstance(r, Literal) for r in rx_set]):
                        rx_set=set([Literal(str({r.toPython() for r in rx_set}))])
        else:
            rx_set = r_set
        

        for d in dx_set:
            for r in rx_set:
                d_classes.append(tuple((p,d,r)))
    return d_classes

#list(get_orientable_owl_object_properties(g))

In [18]:
list(g.triples((URIRef("http://www.tkltd.org/ontologies/datamodel#Relationship"),None, None)))

[(rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#Relationship'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')),
 (rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#Relationship'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),
  rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#Definition')),
 (rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#Relationship'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('Relationship', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))]

In [19]:
list(g.triples((URIRef("http://www.tkltd.org/ontologies/datamodel#ConceptClass"),None, None)))

[(rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#ConceptClass'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')),
 (rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#ConceptClass'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),
  rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#Class')),
 (rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#ConceptClass'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('Concept')),
 (rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#ConceptClass'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#definition'),
  rdflib.term.Literal('A concept in this context is a high level class describing some concept, idea or thing that is of interest to the business. Concepts form the nodes of a conceptual data model. Each model will consist 

In [20]:
lst_dplist = list(get_orientable_owl_object_properties(g, get_concrete_owl_data_properties(g)))
dptypes = dict()
nodes=dict()
processed_lst_dplist=[]
for l,s,t in lst_dplist:
    if isinstance(t, URIRef):
        if t not in dptypes:
            dptypes[t]=1
        else:
            dptypes[t]=dptypes[t]+1
        node_key = f"{str(t.toPython())}_{dptypes[t]}"
        nodes[node_key]=(t,"hexagon", "green")
    else:
        node_key=t.toPython()
        nodes[node_key]=(t.toPython(),"rectangle", "red")
    processed_lst_dplist.append((l,s,node_key))


In [21]:

resolvers = { "dcterms" : "http://purl.org/dc/terms/", 
              "dc" : "http://purl.org/dc/elements/1.1/",
              "disco":"http://www.tkltd.org/ontologies/discourse#", 
              "serial":"http://www.tkltd.org/ontologies/serialization#",
              "dm" : "http://www.tkltd.org/ontologies/datamodel#",
              "rdfs" : "http://www.w3.org/2000/01/rdf-schema#",
              "rdf" : "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
              "xsd" : "http://www.w3.org/2001/XMLSchema#", 
              "skos" : "http://www.w3.org/2004/02/skos/core#",
             
            }



In [22]:
list(get_concrete_owl_classes(g))

[rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#AbstractClass'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#Attribute'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#Class'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#CompoundAttribute'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#ConceptClass'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#ConceptualDataModel'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#Context'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#Definition'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#FunctionalIdentifier'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#LogicalDataModel'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#LogicalEntityClass'),
 rdflib.term.URIRef('http://www.tkltd.org/ontologies/datamodel#MasterDataModel'),
 rdflib.term.URIRef('http://www.tkl

In [23]:
dict_to_html_table(get_node_literals(g, URIRef('http://www.tkltd.org/ontologies/datamodel#ConceptClass'), resolvers))

'<table width="95%"><tr>\n    <th width="20%">Key</th><th width="80%">Value</th>\n    </tr>\n    <tr><td>skos:definition</td><td>A concept in this context is a high level class describing some concept, idea or thing that is of interest to the business. Concepts form the nodes of a conceptual data model. Each model will consist of a unique set of concepts - so if two models reference a "user" concept, there would exist two versions of this, one per model - enabling an independence of definition.</td></tr><tr><td>rdfs:label</td><td>Concept</td></tr>\n    </table>'

In [24]:
nx_g = nx.MultiDiGraph()
ents = set()

for n in list(get_concrete_owl_classes(g)):
    nx_g.add_node(n, label=get_node_label(g,n,resolvers), shape="circle", click=dict_to_html_table(get_node_literals(g, n, resolvers)))
    
for k,v in nodes.items():
    nx_g.add_node(k, label=resolve_uri_label(v[0], resolvers), shape=v[1], color=v[2], click=dict_to_html_table(get_node_literals(g, k, resolvers)))

for s,t in list(get_class_subclass_relations(g)):
    nx_g.add_edge(s,t,label="owl:subClassOf", color="grey")
    
for l,s,t in list(get_orientable_owl_object_properties(g, get_concrete_owl_object_properties(g))):
    nx_g.add_edge(s,t,label=get_node_label(g,l,resolvers),color="blue")

for l,s,t in processed_lst_dplist:
    nx_g.add_edge(s,t,label=get_node_label(g,l,resolvers),color="red")


In [25]:
gv.convert.networkx_to_gjgf(nx_g)

{'graph': {'nodes': {'http://www.tkltd.org/ontologies/datamodel#AbstractClass': {'label': 'Abstract',
    'metadata': {'shape': 'circle',
     'click': '<table width="95%"><tr>\n    <th width="20%">Key</th><th width="80%">Value</th>\n    </tr>\n    <tr><td>skos:definition</td><td>An Abstract is a type of class used to represent a single abstracted version of classes that might be present across multiple models or domains. Its use comes about as a result from the assumption that every class is unique to its given model. Multuple model-level classes are mapped to a single Abstract class inheriting all the model-level details.</td></tr><tr><td>rdfs:label</td><td>Abstract</td></tr>\n    </table>'}},
   'http://www.tkltd.org/ontologies/datamodel#Attribute': {'label': 'Attribute',
    'metadata': {'shape': 'circle',
     'click': '<table width="95%"><tr>\n    <th width="20%">Key</th><th width="80%">Value</th>\n    </tr>\n    <tr><td>rdfs:label</td><td>Attribute</td></tr>\n    </table>'}},
  

In [26]:
gv.d3(gv.convert.networkx_to_gjgf(nx_g), 
      node_label_data_source='label',
      show_edge_label=True,
      edge_label_size_factor=0.7,
      edge_label_data_source='label',
      edge_curvature=0.25,
     links_force_strength=0.1, 
     links_force_distance=100, 
     use_collision_force=True, 
     collision_force_radius=25, 
     many_body_force_strength=-500,
     )

In [13]:
gv.three(gv.convert.networkx_to_gjgf(nx_g), 
      node_label_data_source='label',
      show_edge_label=True,
      edge_label_size_factor=0.7,
      edge_label_data_source='label',
      edge_curvature=0.0,
     links_force_strength=0.1, 
     links_force_distance=100, 
     many_body_force_strength=-500,
     )

In [14]:
import vis_owl

In [27]:
gjgf=vis_owl.gen_gjgf_from_owl_model_graph(g)

gv.d3(gjgf, 
      node_label_data_source='label',
      show_edge_label=True,
      edge_label_size_factor=0.7,
      edge_label_data_source='label',
      edge_curvature=0.25,
     links_force_strength=0.1, 
     links_force_distance=100, 
     use_collision_force=True, 
     collision_force_radius=25, 
     many_body_force_strength=-500,
     ).to_html_partial()

'\n  <style>\n    /* Main divisions */\n    #iKpfEtNtHGq6cfeW5-main-div {\n      line-height: normal;\n      box-sizing: content-box;\n      padding: 3px;\n      background-color: white;\n    }\n    #iKpfEtNtHGq6cfeW5-left-div {\n      float: left;\n      line-height: normal;\n      box-sizing: content-box;\n    }\n    #iKpfEtNtHGq6cfeW5-right-div {\n      float: left;\n      height: 100%;\n      display: none;\n      line-height: normal;\n      box-sizing: content-box;\n    }\n    #iKpfEtNtHGq6cfeW5-right-inner-div {\n      padding-left: 5px;\n      padding-right: 2px;\n      overflow-x: hidden;\n      overflow-y: auto;\n      height: 100%;\n      line-height: normal;\n      box-sizing: content-box;\n    }\n\n    /* Graph and details (contained in left-inner-div) */\n    #iKpfEtNtHGq6cfeW5-graph-div {\n      overflow: hidden;\n      resize: vertical;\n      position: relative;\n      border: 1px solid #ccc;\n      border-radius: 4px;\n      box-shadow: 0px 0px 2px rgba(0, 0, 0, 0.2);\